In [1]:
import db_conn
import pandas as pd
import re

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 추출된 전체 약물
추출된 전체 약물 중 식별이 가능한 UNIQUE한 약물만을 추려냈습니다.

In [3]:
def get_total_drugs():
    cur = conn.cursor()
    cur.execute("""
    SELECT DISTINCT(TRIM(d.tt_drug)) AS t_d
    FROM
    (SELECT id, tt_drug
    FROM article_table_sentences_m M,
        UNNEST(STRING_TO_ARRAY(t_drug, ' , ')) AS tt_drug
    WHERE (tt_drug='') is false) as d;
    """)
    return cur.fetchall()

In [4]:
drugs = pd.DataFrame(get_total_drugs())

In [5]:
# drugs.to_csv('./drugs.csv', sep=',')

In [6]:
drugs.columns=['name']

In [7]:
drugs['refined'] = ''
drugs.head()

,name,refined
0,rivastigmine,
1,edoxaban,
2,levofloxacin-non-susceptible,
3,procarbazine,
4,bambuterol,


#### 전처리 전 총 약물의 수는 2323개 입니다

In [8]:
drugs.shape

(2323, 2)

### 2. 전처리

#### A. 약물을 포함하고 있는 구절이 추출 된 경우, 약물 외적인 단어들을 제거합니다

## 선생님께서 need_to_refined_hyj.xlsx파일을 통해 약물 이름이 아니라고 지정해주신 목록을 추가하였습니다.

In [9]:
replace_with_space = ['-long-acting', '-treated', '-producing', '-high', '-involved-field', '-based', '-studies', '-non-susceptible',\
                      '-susceptible', '-dependent', '-inducible', '-independent', '-eligible', '-rescue', '-sensitive', '-dense',\
                      'infarction-', '11c]-', 'log[', '[99mtc]', '-avid', '-subcutaneous', '-coated', '-activatable', '-total', '-corrected', \
                      '-predicted', '-gated', '-redo-surgery', '-group', '-assigned', '-allocated', '-reduced', '-combination', \
                      '-standard', '-associated', '-ischemic', '-resistant', '-naive', '-sparing', 'concentration', '[', ']',\
                      'low-molecular-weight-', '-extended', '-meter', '-stimulated', 'subtotal', '-labeled', '-matching',\
                      '-resistance;', '-maintenance', '-rate', '-facilitated', '-replacement', '-to-renin', '-gits', '-treated',\
                      '-right', '-or-2-h', '-treatment', '-boosting', '-resistance', '-continuation', '-rich', '&#8211',\
                      '-impregnated', '-dispensation', '-pemetrexed', '-correctable', '-carry-on', '-oral', '-intravenous', '-bare-metal', '-dose',\
                      '-equivalent', '-induced', '-positive', '-related', '-stimulating', '-confirmed', '-controlled', '(', '-level',\
                      '-releasing', '-reuptake', '-eluting', '-containing', '-binding', 'vaccines', 'vaccine', 'change', 'vsstandard',\
                      '-sham', '-adjusted', 'post-', '5m-', '20m-', '-provoked', '<3.2', '30-', '-synthase', '-responsive',\
                      '-pa-824', 'formulation-', 'mi-', 'discretion-', 'polychemotherapy', 'chemotherapy', 'seizures-','crack-',\
                      '-myocardial', 'exercise-', '—4803', '-low', 'mothers', 'von willebrand factor', 'others', '—1688', '-13',
                      # 약물 이름이 아닌 경우
                      'food-borne', 'fast-food', 'food-coping', 'grass-pea-food', 'food-systems', 'food-bolus', 'food-specific', \
                      'food-frequency', 'non-food', 'blood plasma', 'medical air', 'tree pollen', 'grass pollen', 
                      # 추가 약물이름이 아닌 경우
                      'post-food', 'tc-sulfur', 'iodine-to-creatinine', 'protein c2', 'tc–sulfur', 'blood-glucose',\
                      'cy5.5-arginine-glycine-aspartate', 'biotin-aacctgaccacagaggagaggc-3', 'n-2-mercaptopropionylglycine', \
                      'biotin-tccctcatccctgccatgt-3', 'animals-l-arg', 'biotin-cacggcacctggcctaaag-3', 'buserelin+cpax14',\
                      # 추가 확인 필요
                      'collagen-sclerotic', 'collagen-septum', 'albumin:creatinine', 'micro-albuminuria','glucose-insulin-potassium', 'oxygen-18', 'albumin<3.3',\
                      'cyclooxygenase-2', '18f-fluorodeoxyglucose', 'cyclo-oxygenase', 
                     ]

In [10]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in drugs.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if refined.strip() == '':
        drugs.drop(ind, inplace=True)
    elif row['name'].strip() != refined.strip():
        drugs.loc[ind]['refined'] = refined
        print(row['name'])
        print(drugs.loc[ind]['refined'])
    else:
        print(row['name'])
    print('===============================')

drugs = drugs.reset_index(drop=True)

rivastigmine
edoxaban
levofloxacin-non-susceptible
levofloxacin 
procarbazine
bambuterol
metyrapone
cyproheptadine
cyclobenzaprine
pseudo-cholinesterase
famotidine
isepamicin
inductionciclosporin
insulin lispro
lamotrigine-high
lamotrigine 
dulaglutide
acidlevamisole
deutetrabenazine
etanercept
bevacizumab.
triamterene
fluorescein
ceftazidime
isoniazidthiacetazone
low-density-lipoprotein c
tamoxifencmf33
aliskiren
azacitidine
flupirtine
oxygenator
hyaluronic acid
belimumab
pomalidomide
exenatide
vinblastine-involved-field
vinblastine 
sulfadoxine-pyrimethamine
heparinglycoprotein
rifampicinpyrazinamide
emtricitabine
1-sotalol
acetylcarnitine
busulfan-based
busulfan 
collagenase
sotalol
fructose
non-secretin
albumin
ezetimibe<0.001
endothelium
others.
 .
carboplatin
gentamycinemetronidazole
hydrochlorothiazide
rosiglitazone
high-density-lipoprotein cholesterol
albendazole.
nadolol
enalapril-studies
enalapril 
treosulfan-tt-fludarabine
canagliflozin
etoposide-21
acarbose–metformin
crizot

albumin 
indometacin.
clopidogrel
pitavastatin
blockersdiazepam
chondroitin sulfate
ambrisentan
buspirone
neostigmine
strontium ranelate
cisplatin-vinorelbine
ketamine
quetiapine
artemether-lumefantrine
roflumilast
meclozine
doxorubicin–cyclophosphamide
azt+3tc+abacavir+nfv
sotalol-rate
sotalol 
alpha-methylnorepinephrine
sevoflurane
cisplatin-gemcitabine
aldosterone
flecainide
oseltamivir-resistant
oseltamivir 
medroxyprogesterone
sulphadoxine-pyrimethamine
benzylpenicillin–gentamicin
rasagiline
sulfasalazine
albiglutide
pitolisant
sorafenib-related
sorafenib 
sodium phosphate
amodiaquine
chlorpromazine
nystatin
maprotiline
capiricetuximab
vedolizumab
desonide
acetate+prednisone
guanethidine
erythropoietins
alemtuzumab
gadolinium-diethylenetriaminepentaacetic acid
furosemide
malelithium
byeptifibatide
alteplase+sk
pasireotide.
bedaquiline-pa-824
bedaquiline 
eosin
ustekinumab.
diamorphine
ajmaline+
tenecteplase
ribavirin.
quetiapine-er
folfiri–cetuximab
nitazoxanide
heparin-redo-surge

meloxicam
glycine-rich
glycine 
nitrendipine.
u-norepinephrine
atropine.
repaglinide
pyronaridine–artesunate
thrombocytes
ethylenediaminetetraacetic acid
miltefosine
risperidone
quinethazone
gemcitabine-carboplatin
paclitaxel-fec
sorafenib
oxide-telmisartan
iodine-131
iodine 1
pre-terlipressin
non-ciclosporin
flutamide
chloroquinesulfadoxine
digitoxin
rivaroxaban
agelithium
inhibitor-ritonavir
rifaximin
trapidil
dronabinol
oxygenation
b-glucose
chloroquine-proguanil.
ropinirole
procainamide+
glutathione
capecitabine.
trichloroethylene
mycophenolate
carbon dioxide
thiosulfate
gabapentinoids
clofarabine
flunitrazepam
l-asparaginase
cocaine-positive
cocaine 
abatacept
post-protamine
 protamine
rituximab-dose-dense
rituximab  
naftidrofuryl
adenosine-insensitive
aldosterone-responsive
aldosterone 
dipyridamole-thallium
amodiaquinesp
nisoldipine
lipoprotein cholesterolmg
dobutamine40
vecuronium
nab-paclitaxel
adenosine-diphosphate
antithrombin iii
teniposide
formoterol
irinotecan->folfox6
b

gefitinib
ketanserin
bortezomib-rituximab
nrti+saquinavir
nicotine-replacement
nicotine 
melphalan
dobutamine30
quinapril
canakinumab
alglucerase
cisplatin-based
cisplatin 
fluorouracil-based
fluorouracil 
tenecteplase-facilitated
tenecteplase 
lipoprotein c
enoxaparin
stavudinedidanosine
pimavanserin
glucose-tolerance
pamidronic acid
ciprofloxacinmeropenem
boceprevir-containing
boceprevir 
reserpine
dipyridamole-atropine
levodopa-carbidopa
clopamide
vancomycin
heparin–gp2b
cimetidine
sodium nitrite
-epinephrine
calcium folinate
riboflavin
losartan
rituximab-containing
rituximab 
cortivazol
flecainide+
ec-docetaxel
labetalol
norfloxacin
pinacidil
isoprenaline.
empagliflozin
esmolol
methoxy polyethylene glycol-epoetin beta
sunitinib
fluorouracil-plus-oxaliplatin
pindolol
artemether–lumefantrine
dihydrocodeine
lopinavir–ritonavir
sodium citrate
aluminium
palivizumab
orlistat
piperacillin
placebomethotrexate
paracetamol
silver nitrate
pa-824-moxifloxacin-pyrazinamide
fasudil
nafcillin
epl

#### 전처리와 약물이 아닌 레코드를 제거하면 2302개가 남습니다

In [11]:
drugs.shape

(2287, 2)

## 선생님께 확인 요청을 드리고 싶은 부분

## 1. albumin:creatinine 이형태로 표기된 애들은 약제가 아니라고 해주셨는데, 그럼 둘을 포함하고 있는 전체를 제외해도 될까요?

## 아니면 albumin, creatinine 둘 중 하나라도 포함하고 있다면 제외해도 될까요?

In [12]:
drugs[drugs.name.str.find('albumin')>=0]

,name,refined
45,albumin,
116,albumin<3.2,albumin
151,microalbumin,
160,albumincreatinine,
379,macroalbuminuria,
628,normoalbuminuric,
654,albumin-to,
709,albumin-adjusted,albumin
772,albuminuria,
783,albumin-corrected,albumin


In [13]:
drugs[drugs.name.str.find('creatinine')>=0]

,name,refined
160,albumincreatinine,
975,albumin–creatinine,
1129,albumin-to-creatinine,
1535,albumin-creatinine,
1820,protein creatinine,


In [14]:
drugs[(drugs.name.str.find('creatinine')>=0) & (drugs.name.str.find('albumin')>=0)]

,name,refined
160,albumincreatinine,
975,albumin–creatinine,
1129,albumin-to-creatinine,
1535,albumin-creatinine,


## 2. collagen을 포함한 경우 제외해도 될까요?

In [15]:
drugs[drugs.name.str.find('collagen')>=0]

,name,refined
41,collagenase,
157,collagen-induced,collagen
205,collagen,
250,collagen-right,collagen
775,collagen:,
1351,collagen.,
1432,pah–collagen,
1998,procollagen,


## 3. blood을 포함한 경우 제외해도 될까요?

In [16]:
drugs[drugs.name.str.find('blood')>=0]

,name,refined
840,blood-oxygen-level-dependent,blood-oxygen
1093,blood-glucose-lowering,ering
1244,bloodglucose,


## 4. glucose를 포함한 경우 제외해도 될까요?

In [17]:
drugs[drugs.name.str.find('glucose')>=0]

,name,refined
144,glucose-dependent,glucose
209,-fluoro-2-deoxy-d-glucose,
215,2-18f-fluoro-2-deoxy-d-glucose-avid,2-18f-fluoro-2-deoxy-d-glucose
293,glucose-or-2-h,glucose
581,glucose],glucose
587,glucose-6-phosphate,
679,glucoseauc,
795,glucose-containing,glucose
828,glucose-confirmed,glucose
869,glucose:,


## 5. oxygen을 포함한 경우 제외해도 될까요?

In [18]:
drugs[drugs.name.str.find('oxygen')>=0]

,name,refined
28,oxygenator,
79,oxygen-dependent,oxygen
188,preoxygenate,
358,oxygen-treatment,oxygen
593,oxygen,
840,blood-oxygen-level-dependent,blood-oxygen
907,oxygen.,
1386,oxygenation,
1874,cyclo-oxygenase-ii,-ii
1899,cyclooxygenase,


## 6. nitrogen을 포함한 경우 제외해도 될까요?

In [19]:
drugs[drugs.name.str.find('nitrogen')>=0]

,name,refined
113,nitrogen,
1221,nitrogen-13,nitrogen
1625,nitrogen-containing,nitrogen


## 7. thrombin을 포함한 경우 제외해도 될까요?

In [20]:
drugs[drugs.name.str.find('thrombin')>=0]

,name,refined
275,thrombin.,
604,thrombin-induced,thrombin
657,thrombin-activatable,thrombin
944,antithrombin,
984,ma-thrombin,
1212,thrombin-antithrombin,
1267,thrombin-stimulated,thrombin
1334,thrombin,
1416,antithrombin iii,
1626,thrombin&#8211,thrombin


## 8. fetoprotein을 포함한 경우 제외해도 될까요?

In [21]:
drugs[drugs.name.str.find('fetoprotein')>=0]

,name,refined
977,α-fetoprotein concentration,α-fetoprotein
2198,α-fetoprotein,


## 9. renin을 포함한 경우 제외해도 될까요?

In [22]:
drugs[drugs.name.str.find('renin')>=0]

,name,refined
216,aldosterone-to-renin,aldosterone
1662,renin-angiotensin-aldosterone,
1905,renin–angiotensin–aldosterone,


## 복합 약물 정제

### 선생님께서 주신 엑셀 파일을 기반으로 정제합니다

In [23]:
replace_excel = pd.read_excel('./need_to_refined_hyj.xlsx', header=None)
replace_excel.columns = ['id', 'key', 'replace_with', '-', '-', '-']
replace_excel = replace_excel[['key', 'replace_with']]
replace_excel.drop_duplicates(inplace=True)
replace_excel.head()

,key,replace_with
0,17-beta-estradiol,17-beta-estradiol
1,17-beta-estradiol-sham,17-beta-estradiol
2,17-hydroxyprogesterone,17-hydroxyprogesterone
3,5-flucytosine,5-flucytosine
4,5-hydroxyindoleacetic acid,5-hydroxyindoleacetic acid


In [24]:
replace_excel.tail()

,key,replace_with
391,zidovudine-lamivudine-nelfinavir,zidovudine + lamivudine + nelfinavir
392,azt+3tc+abacavir+efv,zidovudine lamivudine abacavir efavirenz
393,azt3tcamprenavir,zidovudine lamivudine amprenavir
394,5-fluorouracil,5-fluorouracil
395,methionine-threonine,methionine-threonine


In [25]:
replace_excel.shape

(392, 2)

In [26]:
for ind, row in replace_excel.iterrows():
    for d_ind, d_row in drugs.loc[drugs.name.str.strip()==row['key']].iterrows():
        origin_str = d_row['name'].replace(row['key'], row['replace_with'])
        
        drugs.loc[d_ind]['refined'] = origin_str 

In [27]:
drugs[drugs.refined.str.find('17-beta-estradiol')>=0]

,name,refined
1158,17-beta-estradiol,17-beta-estradiol
1332,17-beta-estradiol-sham.,17-beta-estradiol .


In [28]:
drugs[drugs.refined.str.find('zidovudine lamivudine amprenavir')>=0]

,name,refined
1321,azt3tcamprenavir,zidovudine lamivudine amprenavir


#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

### 선생님께서 피드백 주신데로 non은 제외하였습니다.
### 선생님께서 피드백 주신데로 약물명 전처리에서 non-제거 & resistant, refractory, pre-, -insensitive, -prev, lipoprotein, untreated, anti-, -receptor 추가하였습니다.

In [29]:
non_drug_ad_rel = ['pseudo', 'placebo', 'negative', 'free', 'withdrawal', 'blocking', 'deficient', 'inhibiting',
                  '-prev', '-insensitive', 'resistant', 'refractory', 'pre-', 'lipoprotein', '-untreated', 'anti-',
                  '-receptor', '-tolerance', '-intolerant', 'non-secretin']
#'non-'

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [30]:
for ind, row in drugs.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
        print(row['name'])
        drugs.drop(ind, inplace=True)
        print('===============================')

drugs = drugs.reset_index(drop=True)

pseudo-cholinesterase
low-density-lipoprotein c
non-secretin
high-density-lipoprotein cholesterol
heparin-placebo
donepezilplacebo
salmeterolplacebo
anti-aldosterone
rifampicin-resistant
rituximab-refractory
progesterone-receptor
low-density-lipoprotein cholesterol
pre-sildenafil
cocaine-negative
tetracycline-resistant
pre-abciximab
iodine-deficient
oseltamivir-resistant
pre-carvedilol
erythromycin-resistant
pre-sirolimus-eluting
lipoprotein cholest
nicotine-free
pre-docetaxel
pre-ivermectin
salmeterol+placebo
buserelin+placebo
placebo+prednisone
tuberculin-negative
ganciclovir-resistant
carvedilol-untreated
pre-mibefradil
enalapril-prev
ceftriaxone-resistant
clindamycin-resistant
interferon-intolerant
cisplatin-refractory
pre-terlipressin
adenosine-insensitive
lipoprotein cholesterolmg
pre-albumin
placebodocetaxel
anti-exenatide
methylprednisolone-placebo
bevacizumab-free
pre-propafenone
high-density-lipoprotein c
anti-interferon
anti-evolocumab
pre-dobutamine
placebo-clopidogrel
cefo

#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 2238 남습니다

In [31]:
drugs.shape

(2222, 2)

### 여러 약물이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [32]:
drugs[drugs.name.str.find('+')>=0].shape

(37, 2)

In [33]:
additional_drugs = []

for ind, row in drugs[drugs.refined.str.find('+')>=0].iterrows():
    processed = list(map(lambda x: x.strip(), row['refined'].split('+')))

    if len(processed)>0:
        drugs.loc[ind].refined = processed[0]
        for pro in processed[1:]:

            additional_drugs.append({'name':row['name'], 'refined':pro})
    
for ind, row in drugs[drugs.refined.str.find(',')>=0].iterrows():
    processed = list(map(lambda x: x.strip(), row['refined'].split(',')))

    if len(processed)>0:
        drugs.loc[ind].refined = processed[0]
        for pro in processed[1:]:

            additional_drugs.append({'name':row['name'], 'refined':pro})


drugs = drugs.append(pd.DataFrame(additional_drugs))
drugs.drop_duplicates(inplace=True)

In [34]:
drugs.shape

(2432, 2)

In [35]:
drugs[drugs.name.str.find('taxane-cyclophosphamide')>=0]

,name,refined
206,mono-taxane-cyclophosphamide,taxane
2167,taxane-cyclophosphamide,taxane
8,mono-taxane-cyclophosphamide,cyclophosphamide
203,taxane-cyclophosphamide,cyclophosphamide


In [36]:
drugs[drugs.name.str.find('ritonavir-idv')>=0]

,name,refined
2190,ritonavir-idv,ritonavir
206,ritonavir-idv,indinavir


### 앞 뒤의 -, . 제거

In [37]:
remove_list = ['.', '-', ':', '+', '–', ';', '%']
def preprocess(name):
    if name[0] in remove_list:
        name = name[1:len(name)]
    
    if name[len(name)-1] in remove_list:
        name = name[0:len(name)-1]
    return name
    
drugs.name = drugs.name.map(lambda x: preprocess(x))

#### 추가 전처리 필요없는 약물들은  refined로 입력

In [38]:
spe_chars = dict((re.escape(str(k)), ' ') for k in remove_list)
sep_pattern = re.compile("|".join(spe_chars.keys()))
int_pattern = re.compile('[0-9]')

for ind, row in drugs.iterrows():
    refined = sep_pattern.sub(lambda m: spe_chars[re.escape(m.group(0))], row['name'])
    int_refined = int_pattern.search(row['name'])
    
    if refined.strip() == '':
        drugs.drop(ind, inplace=True)
    elif row['name'].strip() == refined.strip() and int_refined is None:
        drugs.loc[ind]['refined'] = refined
        print(row['name'])
        print(drugs.loc[ind]['refined'])
    print('===============================')

rivastigmine
0                 
0    pyrimethamine
Name: refined, dtype: object


/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


edoxaban
1               
1    cyproterone
Name: refined, dtype: object
procarbazine
3                 
3    pyrimethamine
Name: refined, dtype: object
bambuterol
4       
4    ATG
Name: refined, dtype: object
metyrapone
5              
5    saquinavir
Name: refined, dtype: object
cyproheptadine
6              
6    penicillin
Name: refined, dtype: object
cyclobenzaprine
7              
7    gentamicin
Name: refined, dtype: object
famotidine
8                    
8    cyclophosphamide
Name: refined, dtype: object
isepamicin
9                    
9    idrabiotaparinux
Name: refined, dtype: object
inductionciclosporin
10              
10    irinotecan
Name: refined, dtype: object
insulin lispro
11              
11    bisoprolol
Name: refined, dtype: object
dulaglutide
13                
13    folinic acid
Name: refined, dtype: object
acidlevamisole
14             
14    ritonavir
Name: refined, dtype: object
deutetrabenazine
15             
15    cisplatin
Name: refined, dtype: object
et

cidofovir
107                
107    fluorouracil
Name: refined, dtype: object
nitrogen
108              
108    ifosfamide
Name: refined, dtype: object
dexpramipexole
109               
109    carboplatin
Name: refined, dtype: object
unfractionatedheparin
110             
110    etoposide
Name: refined, dtype: object
piribedil
112              
112    atazanavir
Name: refined, dtype: object
isosorbide mononitrate
113               
113    delavirdine
Name: refined, dtype: object
nefazodone
114              
114    epirubicin
Name: refined, dtype: object
quinidine
115                    
115    cyclophosphamide
Name: refined, dtype: object
metrizamide
116              
116    paclitaxel
Name: refined, dtype: object
bparomomycin
118                
118    pyrazinamide
Name: refined, dtype: object
teicoplanin
119              
119    artesunate
Name: refined, dtype: object
phenoxymethylpenicillin
120               
120    carboplatin
Name: refined, dtype: object
polymyxin b
121          

cefradine
cefradine
oleandomycin
oleandomycin
amantadine
amantadine
phloroglucinol
phloroglucinol
crizotinib
crizotinib
tobramycinceftazidime
tobramycinceftazidime
naproxcinod
naproxcinod
abciximabn
abciximabn
ranibizumab
ranibizumab
theobromine
theobromine
levetiracetam
levetiracetam
dacarbazine
dacarbazine
adrenocorticotropin
adrenocorticotropin
thiamazole
thiamazole
aprotinin
aprotinin
iodinelong
iodinelong
eletriptan
eletriptan
pertussis vaccines
pertussis vaccines
lumiracoxib
lumiracoxib
enalapril
enalapril
amphotericin bmiltefosine
amphotericin bmiltefosine
fusidic acid
fusidic acid
iopanoic acid
iopanoic acid
erythropoietin
erythropoietin
trastuzumab
trastuzumab
venetoclax
venetoclax
estradiol
estradiol
senega
senega
stavudine
stavudine
triptorelin
triptorelin
aspirinclopidogrel
aspirinclopidogrel
fluconazole
fluconazole
sibutramine
sibutramine
benzoyl peroxide
benzoyl peroxide
dexlansoprazole
dexlansoprazole
pravastatin
pravastatin
orticagrelor
orticagrelor
ritonaviraztddi
rito

propanol
bivalirudinglycoprotein
bivalirudinglycoprotein
bivalirudin
bivalirudin
fluvastatin
fluvastatin
metronidazole
metronidazole
tegafurother
tegafurother
amrubicin
amrubicin
sevelamer
sevelamer
agomelatine
agomelatine
opium
opium
bendamustine
bendamustine
amikacin
amikacin
calcium gluconate
calcium gluconate
atenololchlorthalidone
atenololchlorthalidone
moxonidine
moxonidine
amifostine
amifostine
clomipramine
clomipramine
medrogestone
medrogestone
vorinostat
vorinostat
granisetron
granisetron
melphalan
melphalan
venlafaxine
venlafaxine
daunorubicin
daunorubicin
cefamandolemetronidazole
cefamandolemetronidazole
solifenacin
solifenacin
glucose]
glucose]
omeprazole
omeprazole
ethionamide
ethionamide
perphenazine
perphenazine
nintedanib
nintedanib
trichloroethylene
trichloroethylene
irinotecanfluorouracil
irinotecanfluorouracil
panitumumab
panitumumab
oxygen
oxygen
miglustat
miglustat
piroxicam
piroxicam
watchmanwarfarin
watchmanwarfarin
elvitegravir
elvitegravir
nitazoxanide
nitazoxa

ketoprofen
ketoprofen
montelukast
montelukast
oxygen
oxygen
log[testosterone]
log[testosterone]
docetaxel
docetaxel
alteplase
alteplase
methotrexatecytarabine
methotrexatecytarabine
fenoprofen
fenoprofen
clozapine
clozapine
quinine
quinine
gabapentin
gabapentin
valsartan
valsartan
cibenzoline
cibenzoline
epirubicin
epirubicin
bezafibrate
bezafibrate
chlorprothixene
chlorprothixene
milnacipran
milnacipran
calcitriol
calcitriol
simvastatinezetimibe
simvastatinezetimibe
diazepam
diazepam
everolimus
everolimus
cyproterone
cyproterone
pertuzumab
pertuzumab
acenocoumarol
acenocoumarol
furosemidespironolactone
furosemidespironolactone
[tamoxifen]
[tamoxifen]
abiraterone
abiraterone
iomeprol
iomeprol
metforminthiazolidinedione
metforminthiazolidinedione
epoprostenol
epoprostenol
antithrombin
antithrombin
etoposideifosfamidedoxorubicin
etoposideifosfamidedoxorubicin
zotepine
zotepine
hypereosinophilia
hypereosinophilia
rifampicin
rifampicin
etravirine
etravirine
ceftriaxone
ceftriaxone
gemepros

budesonide
dexamethasone
dexamethasone
trametinib
trametinib
pazopanib
pazopanib
fosinopril
fosinopril
galantamine
galantamine
cloxacillin
cloxacillin
protamine
protamine
fluticasone
fluticasone
fingolimod
fingolimod
thrombin
thrombin
basiliximab
basiliximab
sulbactam
sulbactam
ergotamine
ergotamine
naratriptan
naratriptan
protein
protein
oseltamivir
oseltamivir
tedisamil
tedisamil
rilonacept
rilonacept
insects
insects
amsacrine
amsacrine
ciprofloxacin
ciprofloxacin
collagen
collagen
ceritinib
ceritinib
nicotine
nicotine
promethazine
promethazine
meloxicam
meloxicam
nitrendipine
nitrendipine
atropine
atropine
repaglinide
repaglinide
thrombocytes
thrombocytes
ethylenediaminetetraacetic acid
ethylenediaminetetraacetic acid
miltefosine
miltefosine
risperidone
risperidone
quinethazone
quinethazone
sorafenib
sorafenib
flutamide
flutamide
chloroquinesulfadoxine
chloroquinesulfadoxine
digitoxin
digitoxin
rivaroxaban
rivaroxaban
agelithium
agelithium
rifaximin
rifaximin
trapidil
trapidil
drona

ranolazine
ranolazine
ibrutinib
ibrutinib
ranitidine
ranitidine
daclizumab
daclizumab
bevacizumab
bevacizumab
pemetrexed
pemetrexed
tranexamic acid
tranexamic acid
capsaicin
capsaicin
topotecan
topotecan
lomustine
lomustine
tocilizumab
tocilizumab
dydrogesterone
dydrogesterone
lamotrigine
lamotrigine
ezetimibe
ezetimibe
glucoseinsulin
glucoseinsulin
roquinimex
roquinimex
vemurafenib
vemurafenib
gemtuzumab
gemtuzumab
alogliptin
alogliptin
salmeterol
salmeterol
salmeterolroflumilast
salmeterolroflumilast
oxacillin
oxacillin
tiagabine
tiagabine
misoprostol
misoprostol
podophyllotoxin
podophyllotoxin
reteplase
reteplase
iopamidol
iopamidol
phenelzine
phenelzine
idebenone
idebenone
nivolumab
nivolumab
betacarotene
betacarotene
atorvastatin
atorvastatin
nicotinamide
nicotinamide
cabergoline
cabergoline
zopiclone
zopiclone
atorvastatin
atorvastatin
chlormethine
chlormethine
chlorambucil
chlorambucil
eltrombopag
eltrombopag
deserpidine
deserpidine
midodrine
midodrine
romiplostim
romiplostim
sp

praziquantel
praziquantel
oxprenolol
oxprenolol
antialdosterone
antialdosterone
glucose
glucose
chloramphenicol
chloramphenicol
lorazepam
lorazepam
tamoxifen
tamoxifen
interferon
interferon
amrinone
amrinone
zonisamide
zonisamide
danaparoid
danaparoid
melarsoprol
melarsoprol
oxytocin
oxytocin
polidocanol
polidocanol
clonidine
clonidine
filgrastim
filgrastim
aprindine
aprindine
doxazosin
doxazosin
teicoplanin
teicoplanin
ethambutol
ethambutol
oestradiol
oestradiol
lenalidomide
lenalidomide
rolapitant
rolapitant
oestrone
oestrone
alirocumab
alirocumab
glycopyrronium bromide
glycopyrronium bromide
mipomersen
mipomersen
rosuvastatin
rosuvastatin
carisbamate
carisbamate
zidovudine
zidovudine
fosaprepitant
fosaprepitant
streptokinase
streptokinase
ticagrelor
ticagrelor
ajmaline
ajmaline
gliclazide
gliclazide
hydroxyethylstarch
hydroxyethylstarch
gefitinib
gefitinib
ketanserin
ketanserin
melphalan
melphalan
quinapril
quinapril
canakinumab
canakinumab
alglucerase
alglucerase
enoxaparin
enoxapa

## 정체된 목록입니다.

In [39]:
drugs.loc[drugs.refined.str.strip()!=''].to_csv('./refined.csv', sep=',')

### 추가 확인 요청 데이터

In [40]:
# + 등으로 약물이 나눠진 경우
drugs[(drugs['refined'] == '') & (drugs.name.str.find('+')>=0)]

,name,refined
415,rads+prednisone,


In [41]:
drugs[drugs['refined'] == ''].shape

(182, 2)

In [44]:
drugs[drugs['refined'] == '']

,name,refined
0,rivastigmine,
1,edoxaban,
3,procarbazine,
4,bambuterol,
5,metyrapone,
6,cyproheptadine,
7,cyclobenzaprine,
8,famotidine,
9,isepamicin,
10,inductionciclosporin,


In [42]:
drugs[drugs['refined'] == ''].to_csv('./need_to_refined.csv', sep=',')

In [43]:
drugs.describe()

,name,refined
count,2432,2432
unique,2032,1553
top,rituximab-chop,
freq,5,182
